In [ ]:
import drjit as dr
import mitsuba as mi
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import numpy as np
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2

In [ ]:
mi.set_variant('cuda_ad_rgb')
from mitsuba import ScalarTransform4f as T


In [ ]:
scene_both = mi.load_file("Scene_Data/cbox_lego_both.xml")
view_both = mi.render(scene_both, spp=128)
mi.Bitmap(view_both).write('both.exr')
both = cv2.imread('both.exr', cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)

plt.axis("off")
plt.imshow(cv2.cvtColor(both** (1.0/2.2), cv2.COLOR_BGR2RGB))


In [ ]:
scene_left = mi.load_file("Scene_Data/cbox_left.xml")
view_left = mi.render(scene_left, spp=1024)
mi.Bitmap(view_left).write('left.exr')
left = cv2.imread('left.exr', cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)

plt.axis("off")
plt.imshow(cv2.cvtColor(left** (1.0/2.2), cv2.COLOR_BGR2RGB))

In [ ]:
scene_right = mi.load_file("Scene_Data/cbox_right.xml")
view_right = mi.render(scene_right, spp=1024)
mi.Bitmap(view_right).write('right.exr')
right = cv2.imread('right.exr', cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)

plt.axis("off")
plt.imshow(cv2.cvtColor(right** (1.0/2.2), cv2.COLOR_BGR2RGB))


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

axs[0].imshow(cv2.cvtColor(both**(1.0/2.2), cv2.COLOR_BGR2RGB))
axs[0].set_title("both lights are on")

axs[1].imshow(cv2.cvtColor(left**(1.0/2.2), cv2.COLOR_BGR2RGB))
axs[1].set_title("left light is on")

axs[2].imshow(cv2.cvtColor(right**(1.0/2.2), cv2.COLOR_BGR2RGB))
axs[2].set_title("right light is on")

plt.show()


In [ ]:
def load_sensor(r, theta, phi, pts):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = pts + T.rotate([0, 0, 1], phi).rotate([0, 1, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 39.37,
        'to_world': T.look_at(
            origin=origin,
            target=pts,
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
            'sample_count': 16
        },
        'film': {
            'type': 'hdrfilm',
            'width': 1024,
            'height': 1024,
            'rfilter': {
                'type': 'tent',
            },
            'pixel_format': 'rgb',
        },
    })

In [ ]:
radius = 5
phis = [30*(i) for i in range(6)]
# phis = [0]
thetas = [30*(j) for j in range(6)]
# thetas = [90]
pts = [[1.2,0.8,1]]

sensors = []
for pt in pts:
    for phi in phis:
        for theta in thetas:
            sensors.append(load_sensor(radius,theta,phi,pt))
sensor_count = len(sensors)

In [ ]:
radius = 8
phis = [20*(i) for i in range(9)]
# phis = [0]
thetas = [30*(j) for j in range(6)]
# thetas = [90]
pts = [[1.2,-1,1]]

for pt in pts:
    for phi in phis:
        for theta in thetas:
            sensors.append(load_sensor(radius,theta,phi,pt))
sensor_count = len(sensors)

In [ ]:
sensor_count

In [ ]:
current_dir = os.getcwd()

In [ ]:
current_dir

both image


In [ ]:
scene_both = mi.load_file("Scene_Data/cbox_lego_both.xml")
images_both = [mi.render(scene_both, spp=1024, sensor=sensor) for sensor in sensors]



In [ ]:
num_images = len(images_both)
batch_size = 45
num_batches = num_images // batch_size

for batch in range(num_batches):
    start_index = batch * batch_size
    end_index = start_index + batch_size

    fig = plt.figure(figsize=(16,16))
    fig.subplots_adjust(wspace=0, hspace=0)

    for i, image_index in enumerate(range(start_index, end_index)):
        ax = fig.add_subplot(8,6, i + 1)
        ax.imshow(images_both[image_index] ** (1.0 / 2.2))
        plt.axis("off")

    plt.show()


In [ ]:
folder_name = current_dir+"/Data/Multicam/Lego/Both/images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
for i, sensor in enumerate(sensors):
    os.chdir(folder_name)
    file_name_png = f"frame_{i+1:05d}.png"
    file_path_png = os.path.join(folder_name, file_name_png)
    mi.util.write_bitmap(file_path_png, images_both[i])
    os.chdir(current_dir)
os.chdir(current_dir)    


In [ ]:
print("Done")

right image:

In [ ]:
scene_right = mi.load_file("Scene_Data/cbox_right.xml")
images_right = [mi.render(scene_right, spp=1024, sensor=sensor) for sensor in sensors]

In [ ]:
num_images = len(images_right)
batch_size = 45
num_batches = num_images // batch_size

for batch in range(num_batches):
    start_index = batch * batch_size
    end_index = start_index + batch_size

    fig = plt.figure(figsize=(16,16))
    fig.subplots_adjust(wspace=0, hspace=0)

    for i, image_index in enumerate(range(start_index, end_index)):
        ax = fig.add_subplot(9,5, i + 1)
        ax.imshow(images_right[image_index] ** (1.0 / 2.2))
        plt.axis("off")

    plt.show()


In [ ]:
# folder_name = current_dir+"/Data/right/files_EXR"
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
# for i, sensor in enumerate(sensors):
#     os.chdir(folder_name)
#     phi_value = phis[i // len(thetas)]
#     theta_value = thetas[i % len(thetas)]
#     file_name_exr = f"{phi_value}_{theta_value}.exr"
#     file_path_exr = os.path.join(folder_name, file_name_exr)
#     mi.Bitmap(images_right[i]).write(file_name_exr)
#     # print(i,file_name_exr, file_path_exr)
# os.chdir(current_dir)    


In [ ]:
folder_name = current_dir+"/Data/Multicam/Lego/Right/images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
for i, sensor in enumerate(sensors):
    os.chdir(folder_name)
    file_name_png = f"frame_{i+1:05d}.png"
    file_path_png = os.path.join(folder_name, file_name_png)
    mi.util.write_bitmap(file_path_png, images_right[i])
    os.chdir(current_dir)
os.chdir(current_dir)    


In [ ]:
scene_left = mi.load_file("Scene_Data/cbox_left.xml")
images_left = [mi.render(scene_left, spp=1024, sensor=sensor) for sensor in sensors]


In [ ]:
num_images = len(images_left)
batch_size = 45
num_batches = num_images // batch_size

for batch in range(num_batches):
    start_index = batch * batch_size
    end_index = start_index + batch_size

    fig = plt.figure(figsize=(16,16))
    fig.subplots_adjust(wspace=0, hspace=0)

    for i, image_index in enumerate(range(start_index, end_index)):
        ax = fig.add_subplot(9,5, i + 1)
        ax.imshow(images_both[image_index] ** (1.0 / 2.2))
        plt.axis("off")

    plt.show()


In [ ]:
# folder_name = current_dir+"/Data/Manual_cart/left/files_EXR"
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
# for i, sensor in enumerate(sensors):
#     os.chdir(folder_name)
#     phi_value = phis[i // len(thetas)]
#     theta_value = thetas[i % len(thetas)]
#     file_name_exr = f"{phi_value}_{theta_value}.exr"
#     file_path_exr = os.path.join(folder_name, file_name_exr)
#     mi.Bitmap(images_left[i]).write(file_name_exr)
#     # print(i,file_name_exr, file_path_exr)
# os.chdir(current_dir)    


In [ ]:
folder_name = current_dir+"/Data/Multicam/Lego/Left/images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
for i, sensor in enumerate(sensors):
    os.chdir(folder_name)
    file_name_png = f"frame_{i+1:05d}.png"
    file_path_png = os.path.join(folder_name, file_name_png)
    mi.util.write_bitmap(file_path_png, images_left[i])
    os.chdir(current_dir)
os.chdir(current_dir)    
